# Magnetic Data processing

In this notebook you will undertake the data processing steps that are necessary in order to produce a magnetic anomaly map

## Importing external libraries

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import datetime
from pathlib import Path
from scipy import interpolate
import pyproj
import warnings
warnings.filterwarnings("ignore")

## Importing and visualizing data

The next cells will load the data. 
You will need change the file-path as well as your student id

In [ ]:
# Enter the path to your data folder here
filename = Path(r'C:\Users\julik\sciebo\Master\Geophyphox\Excursion 2022\Student Data\Raw') 

# Enter the student_id you used in the field
student_id = "300724"  

In [ ]:
# Reading the dataset
data = pd.read_csv(os.path.join(filename, "mag_data_"+str(student_id)+".csv"))

# Renaming some columns for easier coding later on 
data = data.rename(columns={'mag_abs':'field'})

# The magnetic field values are stored in µT. Since the convention is to display data in nT, we modify accordingly
data['field'] *= 1000

# Projecting coordinates
projection = pyproj.Proj(proj='utm',zone=32, lat_ts=data['lat'].mean()) #'merc',proj='utm',zone=32,ellps='WGS84'
# Project the dataset coordinates:
easting, northing = projection(np.array(data['lon']), np.array(data['lat']))

data['x'] = easting
data['y'] = northing

In [ ]:
# Taking a first look at the dataset
data.head()

In [ ]:
# We can also call some quick summary statistics in order to get a first feel for our data
data.describe()

The time values in the dataset refer to the elapsed time in seconds since the experiment started. Since we want to combine all phones' measurements later on, it is necessary to normalise the time values to some standard reference time, like 0:00 on the measurement date. The absolute start time of the experiment is saved as [unix-timestamp](https://www.unixtimestamp.com). This timestamp can be converted.

In [ ]:
# Adding a new column to the dataset in which the normalized time is stored.
data["times"] = np.zeros(len(data))
for count,unix in enumerate(data["unix_time"].values):
    t_0_str = datetime.datetime.fromtimestamp(int(unix)).strftime('%Y-%m-%d %H:%M:%S')
    t_0_str = t_0_str[11:]
    t_0 = int(t_0_str[:2])*3600 # Converting hours to seconds
    t_0 += int(t_0_str[3:5])*60 # Converting minutes to seconds
    t_0 += float(t_0_str[6:]) # Adding remaining seconds
    data["times"].iloc[count]=t_0

If, for some reason, one row in the dataset is incomplete (i.e. missing coordinate values), we can not usefully interpret said datapoint. In the next step, we remove all rows that contain missing, or NaN, values. In the dataframe description below, every column should then have the same amount of entries

In [ ]:
# Removin NaN values
data.dropna(inplace=True)
data.describe()

In [ ]:
# Plotting all datapoints in order of appearance in the dataset
fig, ax = plt.subplots(figsize=(12, 8))
nData = len(data['field'])
ax.scatter(data['times'], data['field'])
ax.set(xlabel='Measurement Time [s]', ylabel='B [nT]', title=f'Original dataset with {nData} measurements.');

In [ ]:
# Plotting the collected data spatially
fig, ax = plt.subplots(figsize=(12, 8))

# Using the temporally corrected anomalies
im = ax.scatter(data['x'], data['y'], c=data['field'])
ax.set(xlabel='Easting', ylabel='Northing', title='Collected data')
ax.set_xticklabels(['{:}'.format(int(_)) for _ in ax.get_xticks().tolist()]) # Fixes x-axis tick labels
ax.set_yticklabels(['{:}'.format(int(_)) for _ in ax.get_yticks().tolist()]) # Fixes y-axis tick labels
plt.colorbar(im, ax=ax, label='Magnetic field [T]');

## Getting accurate altitude values

The altitudes recorded with your phones' GPS are likely imprecise. We can get accurate altitude values from a DEM of the survey area:
Enter the path where the DEM file is located on your system.

In [ ]:
# Loading the DEM
dem_location = r'C:\Users\julik\sciebo\Master\Geophyphox\Excursion 2022\DEM\Dürwiss_combined.txt'
dem = pd.read_csv(dem_location, sep=' ', names=['easting', 'northing', 'altitude'])

In [ ]:
z_dem = []

for _x, _y in zip(data['x'], data['y']):
    _x = round(_x, 0)
    _y = round(_y, 0)
    mask = np.logical_and(dem['easting'] == _x, dem['northing'] == _y)
    p = dem.loc[mask]
    z_dem.append(p['altitude'].values[0].copy())
    
data['z_dem'] = z_dem

## Handling Outliers

We can now search the dataset for any outliers. There are many ways of defining and identifying outliers, here we will define a threshold value above or below which we want to discard any values. The median of the data will be our point of reference, and any field value lying above or below a predefined range around the median will be discarded.

In [ ]:
# Here we set the unilateral range around the median, which defines the window of values we want to keep
rangespan = 1300
median_data = data['field'].median()

# Creating a vector with the absolute differences between each field value and the median
diff_median = np.abs(data['field'] - median_data)

# Storing all datapoints that do not meet the outlier criterion
data_outlier = data.loc[diff_median < rangespan]

# This new dataframe is now void of outliers

In [ ]:
# Now we can plot the dataset with outliers removed
fig, ax = plt.subplots(figsize=(12, 8))

nDataOutlier = len(data_outlier['field'])
ax.scatter(data['times'], data['field'])
ax.hlines(median_data, data['times'].min()-500, data['times'].max()+500, color='purple', label='Data median')
ax.hlines(median_data+rangespan, data['times'].min()-500, data['times'].max()+500,
          linestyles='dashed', color='red', label='Upper outlier bound')
ax.hlines(median_data-rangespan, data['times'].min()-500, data['times'].max()+500,
          linestyles='dashed', color='blue', label='Lower outlier bound')
ax.set(xlabel='Measurement Time [s]', ylabel='B [nT]',
       title=f'Outlier removed dataset with {nDataOutlier} measurements, {nData-nDataOutlier} measurements were discarded');
ax.legend()
#plt.xticks(ticks=range(0, len(times_outlier), 200), labels=times_outlier[::200], rotation=45);

## Data Correction

To obtain magnetic anomalies, we need to extract the anomal part of the data. This can be done by subtracting a calculated (or measured) magnetic field value from the data. Another method is to also correct for the temporal drift which naturally occurs, for which measurements at a base station are needed. In our case, we can also use the smartphone measurements taken at the reference location as base station data. Using these data will also eliminate any measurement deviations that occur between smartphones.

Note that only one appropriate method must be chosen.

## Getting Base Station Data

If we want to use smartphone measurements as base station values, we need to get the values that were taken at the reference point. Since GPS measurements are not entirely precise, we define a tolerance range around which we will accept measurements as base station values. Of course we also have to specify the reference points' coordinates.

In [ ]:
# Defining the reference location and tolerance
# Using first measurement locaion as default
refPointX = data_outlier['x'][0]
refPointY = data_outlier['y'][0]

# Defining the tolerance
tolX = 15
tolY = 15

refMinX, refMaxX = (refPointX-tolX, refPointX+tolX)
refMinY, refMaxY = (refPointY-tolY, refPointY+tolY)

maskX = np.logical_and(data_outlier['x'] < refMaxX, data_outlier['x'] > refMinX)
maskY = np.logical_and(data_outlier['y'] < refMaxY, data_outlier['y'] > refMinY)
maskAll = np.logical_and(maskX, maskY)

data_base = data_outlier.loc[maskAll]
nDataBase = len(data_base['x'])

# Plotting reference measurements
fig, ax = plt.subplots(figsize=(12, 8))

im = ax.scatter(data_outlier['x'], data_outlier['y'], c=data_outlier['field'], clip_on=True, label='Measurements')
ax.set(xlabel='Easting', ylabel='Northing', title=f'Magnetic field map with {nDataBase} reference points.')
refArea = mpl.patches.Rectangle((refMinX, refMinY), width=refMaxX-refMinX, height=refMaxY-refMinY,
                               fill=False, label='Reference point selection')
ax.set_yticklabels(['{:}'.format(int(_)) for _ in ax.get_yticks().tolist()]) # Fixes y-axis tick labels
ax.add_artist(refArea)
im_ref = ax.scatter(refPointX, refPointY, c='r', zorder=3, label='Reference point')
plt.colorbar(im, ax=ax, label='Magnetic field [nT]')
ax.legend(handles=[im, im_ref, refArea]);

If the reference area includes less than 2 points, increase the tolerance. Once the base station values are obtained, they can be used in the time drift correction. Set the variable below to 'temporal' in order to use this type of corrrection. For a theoretical reference correction you can also use enter 'reference'.

## Altitude correction 



There is an altitude dependency of the magnetic field. This correction is undertaken before any other calculations are done, and will be done on the basis of a digital elevation model (DEM) of the area.

In [ ]:
# Defining reference height for altitude correction
# Using first measurement location as reference
RefHeight = data_base['z_dem'][0]
dH = data_outlier['z_dem'] - RefHeight
R = RefHeight + 6371000 # (earth radius in m)
field_altitude = data_outlier['field']*(1+(dH/R))**(-3)
data_outlier['field_alt'] = field_altitude


In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(data_outlier['times'], data_outlier['field'], '--', label='Raw data')
ax.plot(data_outlier['times'], data_outlier['field_alt'], label='Altitude corrected data')
ax.set(xlabel='Measurement time [s]', ylabel='Magnetic field B [nT]')
ax.legend()


In [ ]:
data_outlier


## Determining the anomalies

In [ ]:
# Select here how you want to calculate anomalies: 'reference' or 'temporal'
anomaly_method = 'reference'

### Time drift correction

The earth's magnetic field intensity changes throughout the day. If there is data available from a base station, the temporal drift can be measured and subtracted from the measured data. The base station data then also directly serves as a total field reference value, which means that the resulting values represent the magnetic anomaly relative to the base station location.
The anomaly can also be calculated by using a local theoretical reference value, which is demonstrated in the next section.

In [ ]:
if anomaly_method == 'temporal':
    # In order to get basestation values for the exact times at which our actual measurements were taken,
    # we resample the basestation data. This is done via linear interpolation.

    baseline_interpolator = interpolate.interp1d(data_base['times'], data_base['field'], kind='linear',
                                                 fill_value='extrapolate')
    baseline_field = baseline_interpolator(data_outlier['times'])

    field_anomaly = data_outlier['field'] - baseline_field
    
    # Adding anomalies to dataframe
    data_outlier['anomaly'] = field_anomaly
    
    # Plotting the results

    fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(12,8), sharex=True)
    
    ax1.scatter(data_outlier['times'], data_outlier['anomaly'])
    ax1.set(xlabel='Measurement Time [h]', ylabel='B [nT]', title='Temporally corrected magnetic anomalies')
    
    ax0.scatter(data_base['times'], data_base['field'], label='Base station data')
    ax0.plot(data_outlier['times'], baseline_field, 'k', label='Interpolated time drift')
    ax0.set(title='Base station measurements', ylabel='Magnetic field B [nT]')
    ax0.legend();

### Substracting Reference Fiel
If there is no base station data available that can act as a local reference value, a theoretical backround value can be determined. Models for the earth's theoretical magnetic field are available, such as the [World Magnetic Model](http://geomag.bgs.ac.uk/data_service/models_compass/wmm_calc.html), which calculate the theoretical field value B for a given location. This can then be used to calculate deviations between our measurements and the 'ideal' value, returning the desired anomaly without accounting for any temporal trends.

In [ ]:
if anomaly_method == 'reference':
    # Here we define the background value for the correction, taken from literature or a suitable model
    reference = data_base['field'].mean() # nT

    field_anomaly = data_outlier['field'] - reference
    data_outlier['anomaly'] = field_anomaly
    # Plotting the results

    fig, ax = plt.subplots(figsize=(12,8))

    ax.scatter(data_outlier['times'], data_outlier['anomaly'])
    ax.set(xlabel='Measurement Time [s]', ylabel='B [nT]', title='Theoretical magnetic anomalies')

## Plotting all data

We can now visualize all collected data. Starting with a scatterplot:

In [ ]:
# Basic Anomaly map
fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(14, 12))

vmin = -np.max(np.abs(data_outlier['anomaly']))
vmax = np.max(np.abs(data_outlier['anomaly']))

# Using the temporally corrected anomalies
im0 = ax0.scatter(data_outlier['x'], data_outlier['y'], c=data_outlier['field'])
namegroups = data_base.groupby('student_id_list')
for name, group in namegroups:
    ax1.plot(group['times'], group['field'], '-o', label=name)
ax1.legend()
im2 = ax2.scatter(data_outlier['x'], data_outlier['y'], c=data_outlier['anomaly'],
                  vmin=vmin, vmax=vmax, cmap="RdBu_r")

ax0.set(xlabel='Easting', ylabel='Northing', title='Outlier & altitude corrected data')
ax1.set(xlabel='Measurement time [s]', ylabel='Magnetic field B [nT]', title='Base station data')
ax2.set(xlabel='Easting', ylabel='Northing', title='Field anomaly map')
plt.colorbar(im0, ax=ax0, label='Magnetic field B [nT]');
#plt.colorbar(im1, ax=ax1, label='Magnetic field B [nT]');
plt.colorbar(im2, ax=ax2, label='Magnetic anomaly [nT]');

ax0.set_yticklabels(['{:}'.format(int(x)) for x in ax0.get_yticks().tolist()]) # Fixes y-axis tick labels
ax2.set_yticklabels(['{:}'.format(int(x)) for x in ax2.get_yticks().tolist()]) # Fixes y-axis tick labels

plt.tight_layout()

## Saving the results

As a last step, we save our results. There are a multitude of ways and fileformats to achieve this. Since we want to store numerical data, [Numpy's save function](https://numpy.org/doc/stable/reference/generated/numpy.savez.html) is a fast and effective method to achieve this. We save the calculated anomalies as well as all data processing steps in between.

In [ ]:
# Save the data. Give your datafile a name so it can be distinguished from the others.

fileName = 'results_magnetic_' + student_id + '.csv'
baseFileName = 'results_magneticBASE_' + student_id + '.csv'

#Enter path where data should be exported too
folder = Path(r'C:\Users\julik\sciebo\Master\Geophyphox\Excursion 2022\Student Data\Processed')
data_outlier['name'] = [student_id]*len(data_outlier)

data_base['name'] = [student_id]*len(data_base)
data_outlier.to_csv(folder / fileName)
data_base.to_csv(folder/ baseFileName)